In [241]:
import requests
from bs4 import BeautifulSoup
from Module.selenium_driver import webdriver_control
import sys
from difflib import SequenceMatcher
from config import config
import pymysql
import pandas as pd
from Module.utility import result2csv
import datetime as dt

In [230]:
# DB 연동
def connect_db_rds():
    conn = pymysql.connect(
        host = config.RDS_config['host']
        , user = config.RDS_config['user']
        , password = config.RDS_config['password']
        , database = config.RDS_config['database']
        , port = config.RDS_config['port']
        , cursorclass=pymysql.cursors.DictCursor
    )
    return conn

conn_rds = connect_db_rds()

In [233]:
cursor = conn_rds.cursor()
sql = config.query['select_music_list']
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(result)

In [177]:
# 드라이버 경로
DRIVER_PATH = 'D:/'

# 운영체제별 드라이버 선택
platform = sys.platform
# 윈도우일 경우
if platform == 'win32':
    DRIVER_PATH += 'chromedriver.exe'
# 리눅스일 경우
elif platform == 'linux':
    DRIVER_PATH += 'chromedriver_linux.exe'

# Chrome Driver init
chrome = webdriver_control.generate_chrome(driver_path=DRIVER_PATH)

In [178]:
chrome.get('https://www.youtube.com/results?search_query=bts+butter%EB%A7%8C')
time.sleep(0.5)
source = chrome.page_source
soup = BeautifulSoup(source, 'html5lib')

a = soup.findAll('a', id='video-title')
span = soup.findAll('span', id='text')

In [242]:
t = dt.datetime.now()
save_time = t.strftime('%Y_%W')

In [224]:
for i in range(len(df)):
    music_idx = df['music_idx'][i]
    title = df['title'][i]
    artist = df['name'][i]

    result = {
        'music_idx' : music_idx,
        'title' : title,
        'artist' : artist,
        'yt_link1' : None,
        'yt_link2' : None,
    }
    
    result_link = []
    for i in range(len(a)):
        if '시간'  in span[i]['aria-label']:
            continue
        else:
            play_time = int(span[i]['aria-label'].split('분 ')[0])*60 + int(span[i]['aria-label'].split('분 ')[1].replace('초', ''))

            if (play_time <= 360) and (title.upper() in a[i]['title'].upper()):
                href = a[i]['href']
                url = f'https://www.youtube.com{href}'
                result_link.append(url)
        
        if len(result_link) == 2:
            break
    
    if len(result_link) == 2:
        result['yt_link1'] = result_link[0]
        result['yt_link2'] = result_link[1]
    elif len(result_link) == 1:
        result['yt_link1'] == result_link[0]
    else:
        continue
    
    df_result = pd.DataFrame(result)
    result2csv(f'./csv/music_link_{save_time}.csv', df_result)

In [250]:
result_link

['https://www.youtube.com/watch?v=DSEO6BAh3kw',
 'https://www.youtube.com/watch?v=rZI_Nt2mPgA']